In [534]:
import json
import pandas as pd

import MeCab

from gensim.models.doc2vec import Doc2Vec
from gensim.models.doc2vec import TaggedDocument

from sklearn.manifold import TSNE

import plotly.express as px
from datetime import datetime


In [535]:
stories_json = []
with open('./output_stories_0123.json', 'r') as f:
    stories_json = stories_json + json.loads(f.read())

print(len(stories_json))
print(stories_json[1].keys())

839
dict_keys(['index', 'stories_index', 'title', 'broadcasting_date', 'abstracts_list'])


In [536]:
stories_df = pd.DataFrame(stories_json)
stories_df["abstract_joined"] = stories_df["abstracts_list"].str.join(" ")
stories_df["abstract_title_joined"] = stories_df["title"].str.join("") + " " + stories_df["abstracts_list"].str.join(" ")
stories_df.head()

,index,stories_index,title,broadcasting_date,abstracts_list,abstract_joined,abstract_title_joined
0,0325,0,「映画ドラえもん のび太の新魔界大冒険～７人の魔法使い～」,2013年3月15日,[ 何をやってもうまくいかないのび太は、「魔法（まほう）が使えたら便利で楽しくなるにちがいな...,何をやってもうまくいかないのび太は、「魔法（まほう）が使えたら便利で楽しくなるにちがいない...,「映画ドラえもん のび太の新魔界大冒険～７人の魔法使い～」 何をやってもうまくいかないのび...
1,0326,0,「ばくはつコショウ」,2013年4月12日,[ ある朝、ねぼうしたのび太に泣きつかれたドラえもんは、『ばくはつコショウ』を取り出す。この...,ある朝、ねぼうしたのび太に泣きつかれたドラえもんは、『ばくはつコショウ』を取り出す。このコ...,「ばくはつコショウ」 ある朝、ねぼうしたのび太に泣きつかれたドラえもんは、『ばくはつコショ...
2,0326,1,「何が何でもお花見を」,2013年4月12日,[ みんなから家族でお花見に行った話を聞き、うらやましくてしかたのないのび太。さっそくママに...,みんなから家族でお花見に行った話を聞き、うらやましくてしかたのないのび太。さっそくママにた...,「何が何でもお花見を」 みんなから家族でお花見に行った話を聞き、うらやましくてしかたのない...
3,0327,0,「やりクリしてハワイ旅行」,2013年4月26日,[ スネ夫からハワイ旅行をじまんされたのび太は、自分もハワイに行きたいとママにたのもうとする...,スネ夫からハワイ旅行をじまんされたのび太は、自分もハワイに行きたいとママにたのもうとするが...,「やりクリしてハワイ旅行」 スネ夫からハワイ旅行をじまんされたのび太は、自分もハワイに行き...
4,0327,1,「しずかちゃんのはごろも」,2013年4月26日,[ 学校で「天女と羽衣（はごろも）」の劇（げき）をやることになったのび太としずか。劇の最後、...,学校で「天女と羽衣（はごろも）」の劇（げき）をやることになったのび太としずか。劇の最後、天...,「しずかちゃんのはごろも」 学校で「天女と羽衣（はごろも）」の劇（げき）をやることになった...


In [537]:
t = MeCab.Tagger('-d /opt/homebrew/lib/mecab/dic/mecab-ipadic-neologd')
def mecab_tokenizer(text):
    parsed_lines = t.parse(text).split("\n")[:-2]
    surfaces = [l.split('\t')[0] for l in parsed_lines]
    features = [l.split('\t')[1] for l in parsed_lines]
    # 原型を取得
    bases = [f.split(',')[6] for f in features]
    # 品詞を取得
    pos = [f.split(',')[0] for f in features]

    # 各単語を原型に変換する
    token_list = [b if b != '*' else s for s, b in zip(surfaces, bases)]
    # 品詞を絞り込み
    target_pos = ["名詞", "形容詞", "動詞"]
    token_list = [t for t, p in zip(token_list, pos) if p in target_pos]
    # アルファベットを小文字に統一
    token_list = [t.lower() for t in token_list]

    return [ text for text in filter(lambda x: not x.isascii(), token_list)]

wt = MeCab.Tagger('-Owakati')
def mecab_tokenizer_wakachi(text):
    return wt.parse(text).split(" ")

In [538]:
# ストーリーが取得できたものに絞る
stories_df = stories_df[stories_df['abstract_title_joined'].str.len() > 0] 
sentences = stories_df["abstract_title_joined"].apply(mecab_tokenizer_wakachi).apply(set).apply(list)
print(sentences)

0      [する, ため, 地球, しずか, て, ある, スネ, ない, ボックス, 日, とともに...
1      [これ, ぼう, て, ある, こく, ない, さわぎ, その, 目的, だ, こと, \n...
2      [する, パパ, しずか, 楽しかっ, て, ない, 使っ, たのん, 何でも, 型, こと...
3      [する, でき, て, スネ, ない, 日, 給料, 様子, みごと, 1000, こと, ...
4      [ため, しずか, て, よろこび, 天女, 返し, 上がり, だ, ちゃん, 劇, こと,...
                             ...                        
834    [しか, ため, する, プラネタイジリウム, これ, いざ, て, ある, スネ, ない,...
835    [話す, 買っ, パパ, せよ, て, ない, なかなか, 使っ, いれ, 落とし, その,...
836    [する, さそう, せよ, ぼう, て, ほか, 身, 見かけ, スネ, ろう, 的, ホ,...
837    [ため, する, て, 顔, その, ちょうど, だ, さけぶ, 家, \n, かせ, 青ざ...
838    [やって来, しずか, 取りはずし, て, よろこび, 巨大, すごす, ない, そこで, ...
Name: abstract_title_joined, Length: 839, dtype: object


In [539]:
abstract_list = sentences.tolist()
trainings = [TaggedDocument(words = data, tags = [i]) for i,data in enumerate(abstract_list)]
m = Doc2Vec(documents= trainings, dm = 1, window=8, min_count=10, workers=4)


In [540]:
vector_list = m.dv.vectors
tsne = TSNE(n_components=2)
vector_enbedded = tsne.fit_transform(vector_list)
print(vector_enbedded)

/Users/takamirei/.pyenv/versions/3.9.5/lib/python3.9/site-packages/sklearn/manifold/_t_sne.py:795: FutureWarning:

The default initialization in TSNE will change from 'random' to 'pca' in 1.2.

/Users/takamirei/.pyenv/versions/3.9.5/lib/python3.9/site-packages/sklearn/manifold/_t_sne.py:805: FutureWarning:

The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.



[[ -6.731565    6.5610514]
 [-38.483707  -36.9642   ]
 [-39.487648  -35.070164 ]
 ...
 [ 17.411047   19.034323 ]
 [-26.510065  -28.46241  ]
 [-34.317688  -27.648697 ]]


In [541]:
vector_df = pd.DataFrame.from_dict({
    'vector': [v for v in vector_list],
    'vector_x': [v[0] for v in vector_enbedded],
    'vector_y': [v[1] for v in vector_enbedded]
})
stories_df_merged = stories_df.join(vector_df)
stories_df_merged['broadcasting_dt'] = stories_df_merged['broadcasting_date'].apply(lambda d: datetime.strptime(d, '%Y年%m月%d日'))
stories_df_merged['broadcasting_year'] = stories_df_merged['broadcasting_dt'].dt.year
stories_df_merged[["title", "abstracts_list", "stories_index", 'broadcasting_date', "broadcasting_dt", "abstract_joined", 'vector']].reset_index().to_csv("stories_df_merged.csv")


In [542]:
fig = px.scatter(stories_df_merged, x='vector_x', y='vector_y', color='broadcasting_year', hover_data=["index", "title"])
fig.show()

In [543]:
df = px.data.tips()
fig = px.scatter(stories_df_merged, x='vector_x', y='vector_y', hover_data=["index", "title"], facet_row="broadcasting_year", width=800, height=2000)
fig.show()

In [544]:
sampled_df = stories_df_merged.sample(1)
print(sampled_df.head())
sampled_df = sampled_df.rename(columns={"vector": "sampled_vec"})[["sampled_vec"]]


    index  stories_index       title broadcasting_date  \
371  0510              0  「ロボ子が愛してる」         2018年2月2日   

                                        abstracts_list  \
371  [スナック菓子の連続投げ食べを、しずかたちクラスメイトの女の子にほめられ、コーチしてほしいと...   

                                       abstract_joined  \
371  スナック菓子の連続投げ食べを、しずかたちクラスメイトの女の子にほめられ、コーチしてほしいと言...   

                                 abstract_title_joined  \
371  「ロボ子が愛してる」 スナック菓子の連続投げ食べを、しずかたちクラスメイトの女の子にほめられ...   

                                                vector  vector_x  vector_y  \
371  [-0.058293786, 0.04613965, 0.004428133, -0.075...  7.578371  6.398253   

    broadcasting_dt  broadcasting_year  
371      2018-02-02               2018  


In [545]:
stories_df_merged = stories_df_merged.assign(key=1).merge(sampled_df.assign(key=1), on='key').drop('key', axis=1)


In [546]:
from scipy.spatial.distance import cosine

stories_df_merged["cosine_similarity"] =  stories_df_merged.apply(lambda v: 1 - cosine(v["vector"], v["sampled_vec"]), axis=1)
stories_df_merged_sort = stories_df_merged[["title", "abstract_joined", "cosine_similarity"]].sort_values("cosine_similarity", ascending=False)


In [547]:
stories_df_merged_sort

,title,abstract_joined,cosine_similarity
371,「ロボ子が愛してる」,スナック菓子の連続投げ食べを、しずかたちクラスメイトの女の子にほめられ、コーチしてほしいと言...,1.000000
187,「うら山のウサギ怪獣（かいじゅう）」,\r\n 学校の裏山に、０点の答案用紙をかくしに行ったのび太は、ウサギを見つけ、大よろこび。...,0.995171
647,「やどり木で楽しく家出」,ママが作っているカレーに、「ニンジンは入れないで」と注文をつけるのび太。さらに、好ききらいは...,0.994881
698,「月の光と虫の声」,スネ夫が高級デパートで買ってもらったというスズムシの鳴き声を聞いていたのび太、しずか、ジャイ...,0.994729
824,「バトンタッチでおまかせ！自動返送荷札」,スネ夫が持っている人気のマンガ本を早く借りたいというジャイアン。ところが、だれかに貸（か）し...,0.994684
...,...,...,...
95,「怪談ランプ」,\r\n ジャイアンたちと怪談（かいだん）大会をやることになったのび太は、こわい話をしてジャ...,-0.820828
720,「ドラえもん＆パーマン危機一髪！？」,スネ夫の家で、大迫力（はくりょく）のホームシアターをじまんされたのび太。くやしいと泣きつかれ...,-0.845879
3,「やりクリしてハワイ旅行」,スネ夫からハワイ旅行をじまんされたのび太は、自分もハワイに行きたいとママにたのもうとするが...,-0.899825
288,「ドラえもん＆パーマン危機一髪!?」,\r\n スネ夫の家で、大迫力のホームシアターをじまんされたのび太。くやしいと泣きつかれたド...,-0.922572
